In [1]:
import os
import numpy as np
import pandas as pd

from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql.window import *

In [2]:
import pyspark
from pyspark.sql import SparkSession
# import pyspark.pandas as ps

spark = SparkSession.builder.appName('HM-recommendations').getOrCreate()

In [15]:
spark

In [4]:
spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", True)

In [5]:
transaction = spark.read.option('header','true').parquet('../data/HM_parquet/transactions_train.parquet')
articles = spark.read.option('header','true').parquet('../data/HM_parquet/articles.parquet')
customers = spark.read.option('header','true').parquet('../data/HM_parquet/customers.parquet')

In [6]:
trainOneMonth = spark.read.option('header','true').parquet('../data/train_one_month.parquet')
valOneMonth = spark.read.option('header','true').parquet('../data/val_one_month.parquet')

In [7]:
customers.columns

['customer_id',
 'FN',
 'Active',
 'club_member_status',
 'fashion_news_frequency',
 'age',
 'postal_code']

In [8]:
transaction.columns

['t_dat', 'customer_id', 'article_id', 'price', 'sales_channel_id']

In [9]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.ml.feature import StringIndexer


In [ ]:
# 看spark設定
# spark.conf.get("spark.serializer")

In [ ]:
# 改spark buffer 設定
# spark.conf.set("spark.serializer", "org.apache.spark.serializer.KryoSerializer")
# spark.conf.set("spark.kryoserializer.buffer.max", "128m")

In [ ]:
# 將customer_id的字串轉為數字(buffer不夠大)
# indexer = StringIndexer(inputCol="customer_id", outputCol="customer_index")
# indexed = indexer.fit(customers).transform(customers)
# indexed.show()

In [ ]:
def string_to_index(self,transactions,customers,articles):
        """傳入transaction,customers,articles 的 DataFrame，接著取出完整的users和items清單，然後將customer_id和article_id轉為編號，並且map到transaction上形成新的欄位。

        Args:
            transactions ([DataFrame])
            customers ([DataFrame])
            articles ([DataFrame])
            
        Returns:
            transactions ([DataFrame])
        """    
        dfu = customers
        dfi = articles
        ALL_USERS = dfu.select('customer_id').distinct()
        ALL_ITEMS = dfi.select('article_id').distinct()

        user_ids = dict(list(enumerate(ALL_USERS)))
        item_ids = dict(list(enumerate(ALL_ITEMS)))

        user_map = {u: uidx for uidx, u in user_ids.items()}
        item_map = {i: iidx for iidx, i in item_ids.items()}
        # 將落落長的使用者id和商品id轉為編號
        transactions['user_id'] = transactions['customer_id'].map(user_map)
        transactions['item_id'] = transactions['article_id'].map(item_map)

        del dfu, dfi
        return transactions

In [25]:
dfu = customers
ALL_USERS = dfu.select('customer_id').distinct()
user_ids = dict(list(enumerate(ALL_USERS)))
user_map = {u: uidx for uidx, u in user_ids.items()}
transaction.withColumn('user_id')

In [ ]:
als=ALS(maxIter=5,regParam=0.09,rank=25,userCol="customer_id",itemCol="article_id",ratingCol="count",coldStartStrategy="drop",nonnegative=True)
model=als.fit(training)